In [1]:
import os
import cv2
from ultralytics import YOLO
import xgboost as xgb
import pandas as pd
import time
from datetime import datetime

def detect_shoplifting_live(camera_index=0, output_folder="output_video"):
    # Load models
    model_yolo = YOLO(r"/Users/puttu/Desktop/FYP-shoplifting/FYP-shoplifting/best.pt")
    model = xgb.Booster()
    model.load_model(r"/Users/puttu/Desktop/FYP-shoplifting/FYP-shoplifting/model_weights.json")
    
    # Initialize camera with proper settings
    cap = cv2.VideoCapture(camera_index, cv2.CAP_AVFOUNDATION)  # Use AVFoundation for macOS
    if not cap.isOpened():
        # Try different camera indices if default fails
        for i in range(3):
            cap = cv2.VideoCapture(i, cv2.CAP_AVFOUNDATION)
            if cap.isOpened():
                print(f"Found camera at index {i}")
                break
        else:
            print("Error: Could not open any camera.")
            return
    
    # Set reasonable resolution (adjust as needed)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    
    # Get actual camera properties
    fps = int(cap.get(cv2.CAP_PROP_FPS)) if cap.get(cv2.CAP_PROP_FPS) > 0 else 30
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f"Camera resolution: {width}x{height} at {fps} FPS")
    
    # Create output folder
    os.makedirs(output_folder, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_path = os.path.join(output_folder, f"live_detection_{timestamp}.mp4")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    # Alert management
    last_alert_time = 0
    alert_cooldown = 30  # seconds
    
    print("Live detection running... Press 'q' to quit")
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Frame capture error - retrying...")
                time.sleep(0.1)
                continue
            
            # Perform detection
            results = model_yolo(frame, verbose=False, imgsz=640)  # Adjusted size for performance
            annotated_frame = results[0].plot(boxes=False)
            
            suspicious_detected = False
            
            for r in results:
                boxes = r.boxes.xyxy.cpu().numpy()
                conf = r.boxes.conf.cpu().numpy()
                keypoints = r.keypoints.xyn.cpu().numpy()
                
                for i, (box, cf) in enumerate(zip(boxes, conf)):
                    if cf > 0.75:
                        data = {}
                        for j, (x, y) in enumerate(keypoints[i]):
                            data[f'x{j}'] = x
                            data[f'y{j}'] = y
                        
                        # Make prediction
                        df = pd.DataFrame(data, index=[0])
                        dmatrix = xgb.DMatrix(df)
                        pred = (model.predict(dmatrix) > 0.5).astype(int)[0]
                        
                        x1, y1, x2, y2 = map(int, box[:4])
                        if pred == 0:  # Suspicious
                            suspicious_detected = True
                            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (255, 7, 58), 2)
                            cv2.putText(annotated_frame, f'Suspicious ({cf:.2f})', 
                                       (x1, y1-10), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 7, 58), 2)
                        else:  # Normal
                            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (57, 255, 20), 2)
                            cv2.putText(annotated_frame, f'Normal ({cf:.2f})', 
                                       (x1, y1-10), cv2.FONT_HERSHEY_DUPLEX, 0.8, (57, 255, 20), 2)
            
            # Display and record
            cv2.imshow('Live Shoplifting Detection', annotated_frame)
            out.write(annotated_frame)
            
            # Alert if needed
            if suspicious_detected and (time.time() - last_alert_time) > alert_cooldown:
                last_alert_time = time.time()
                print(f"\nALERT: Suspicious activity detected at {datetime.now().strftime('%H:%M:%S')}")
                # Add your email alert function here
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
    except KeyboardInterrupt:
        pass
    
    # Cleanup
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"\nDetection stopped. Recording saved to {output_path}")

# Run with different camera indices if needed
detect_shoplifting_live(camera_index=0)  # Try 0, 1, 2 etc. if default doesn't work

Camera resolution: 1280x720 at 30 FPS
Live detection running... Press 'q' to quit

ALERT: Suspicious activity detected at 12:20:10

Detection stopped. Recording saved to output_video/live_detection_20250412_122008.mp4
